In [100]:
import numpy as np
import tensorflow as tf
from tensorflow.python import keras

In [101]:
from tensorflow.python.keras.preprocessing.image import ImageDataGenerator

train_idg = ImageDataGenerator(rescale=1. / 255.)
test_idg = ImageDataGenerator(rescale=1. / 255.)

train_idg = train_idg.flow_from_directory("./animals/train", target_size=(224, 224), batch_size=32, class_mode="binary")
test_idg = test_idg.flow_from_directory("./animals/test", target_size=(224, 224), batch_size=32, class_mode="binary")

Found 2699 images belonging to 3 classes.
Found 301 images belonging to 3 classes.


In [102]:
from tensorflow.python.keras.applications.vgg16 import VGG16

base_model = VGG16(include_top=False)
base_model.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_8 (InputLayer)         [(None, None, None, 3)]   0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, None, None, 64)    1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, None, None, 64)    36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, None, None, 64)    0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, None, None, 128)   73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, None, None, 128)   147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, None, None, 128)   0     

In [103]:
from tensorflow.python import keras

x = base_model.outputs[0]
x = keras.layers.GlobalAveragePooling2D()(x)
x = keras.layers.Dense(1024, activation=tf.nn.relu)(x)
y_predict = keras.layers.Dense(3, activation=tf.nn.softmax)(x)

model = keras.models.Model(inputs=base_model.inputs, outputs=y_predict)
model

In [104]:
# 冻结VGG模型
for layer in base_model.layers:
    layer.trainable = False

In [105]:
from tensorflow.keras import optimizers
from tensorflow.python.keras.callbacks import ModelCheckpoint

model.compile(optimizer="adam", loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

check = ModelCheckpoint("./model/transfer_{epoch:02d}-{val_accuracy:.2f}.h5", monitor='val_accuracy',
                        save_best_only=True,
                        save_weights_only=True, mode='auto')
model.fit_generator(train_idg, validation_data=test_idg, epochs=5, callbacks=[check])

Epoch 1/5


D:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\keras\engine\training.py:1963: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


85/85 [==============================] - 360s 4s/step - loss: 0.6309 - accuracy: 0.7536 - val_loss: 0.3533 - val_accuracy: 0.8771

In [111]:
from tensorflow.python.keras.models import load_model
from tensorflow.python.keras.preprocessing.image import img_to_array, load_img
from tensorflow.python.keras.applications.vgg16 import preprocess_input

# model.load_weights("./model/transfer_07-0.92.h5")
for i in range(901,933):
    image = load_img("./animals/test/panda/panda_00" + str(i) + ".jpg", target_size=(224, 224))
    image = img_to_array(image)
    image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
    # print(image.shape)

    image = preprocess_input(image)
    predictions = model.predict(image)
    # print(predictions)
    # print(np.argmax(predictions[0]))
    animal_dict = {
        0: "cat",
        1: "dog",
        2: "panda"
    }
    print(animal_dict[np.argmax(predictions[0])])

panda
panda
panda
panda
panda
panda
panda
panda
panda
panda
panda
panda
panda
panda
panda
panda
panda
dog
panda
panda
panda
dog
panda
panda
panda
panda
panda
panda
panda
panda
panda
panda
